In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
# Viz plots
import matplotlib.pyplot as plt
import seaborn as sns
from autorocks.viz import plots_setup


plt.style.use("ggplot")
sns.set_theme(style="ticks", rc={"axes.spines.right": False, "axes.spines.top": False})
sns.set_context("paper")  # , font_scale=1.5, rc={"lines.linewidth": 1.5})
plt.rcParams["svg.fonttype"] = "none"
plt.rcParams["font.family"] = "Arial"
plt.rc("text", usetex=False)
plt.rc("xtick", labelsize="large")
plt.rc("ytick", labelsize="large")
plt.rc("axes", labelsize="large")
plt.rc("pdf", use14corefonts=True)


In [ ]:
# Ok the goal here is to evaluate Branin as a parameteric model 

import torch
import gpytorch
import botorch.posteriors
from botorch.optim import fit
import numpy as np
import pandas as pd

class ParametricMean(gpytorch.means.Mean):
    def __init__(self, input_size, batch_shape=torch.Size()):
        super().__init__()
        self.register_parameter(name="weight_pow", parameter=torch.nn.Parameter(torch.randn(*batch_shape, input_size, 1)))
        self.register_parameter(name="weight_sin", parameter=torch.nn.Parameter(torch.randn(*batch_shape, input_size, 1)))
        self.register_parameter(name="bias_pow", parameter=torch.nn.Parameter(torch.randn(*batch_shape, 1)))
        self.register_parameter(name="bias_sin", parameter=torch.nn.Parameter(torch.randn(*batch_shape, 1)))

    def forward(self, x):
        return (torch.pow(x * self.weight_pow + self.bias_pow, 2) * torch.sin(x * self.weight_sin + self.bias_sin)).squeeze(-1) 


class Node:
    
    def predict(self):
        pass 
    
    def fit(self):
        pass 

class DAG:
    def __init__(self):
        self._nodes = [] 
        
    def add(self, node: Node):
        self._nodes.append(node)
    
    def predict(self):
        pass 
    
    def fit(self):
        for node in self._nodes:
            node.fit()
            
# Then we create a botorch wrapper on top of the DAG            

# BOBN Eye
def negative_log_predictive_density(dag: DAG) -> float :
    """Outputs negative log likelihood of the test data given the predictive distribution"""
    